In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

data_path = "genres/data.json"


def load_data(dataset_path):
    """
    Loads training dataset from json file
    :param data_path (str): Path to json file containing data
    :return X (ndarray): Inputs
    :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    #convert lists into numpy arrays
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    return X, y


def prepare_datasets(test_size, validation_size):

    #load data
    X, y = load_data(data_path)

    #create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=test_size)

    #create train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(
        X_train, y_train, test_size=validation_size)

    return X_train, X_validation, X_test, y_train, y_validation, y_test


def build_model(input_shape):
    """Generate RNN-LSTM model"""

    #create model
    model = keras.Sequential()

    #2 LSTM layers
    model.add(
        keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))

    #dense layer
    model.add(keras.layers.Dense(64, activation='relu'))
    #to mitigate the overfitting issue
    model.add(keras.layers.Dropout(0.3))

    #output layer uses softmax
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model


def predict(model, X, y):

    X = X[np.newaxis, ...]

    #expects a 4d array
    #prediction is a 2D array = [[0.1, 0.2, ...]] 10 different scores
    prediction = model.predict(X)  # X->(1 (# of sample),130,13,1)

    #extract the index with max value
    predicted_index = np.argmax(prediction, axis=1)  #[3] index predicted
    print("Expected index: {}, Predicted index{}".format(y, predicted_index))


if __name__ == "__main__":

    #create train, validation, and test sets
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(
        0.25, 0.2)

    #create network
    input_shape = (X_train.shape[1], X_train.shape[2])  #130,13
    model = build_model(input_shape)

    #compile the network
    optimizer = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer,
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    model.summary()

    #train the model
    history = model.fit(X_train,
                        y_train,
                        validation_data=(X_validation, y_validation),
                        batch_size=32,
                        epochs=30)

    #evaluate the model on test set
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    print("Accuracy on test set is {}".format(test_accuracy))

    #make prediction on a sample
    X = X_test[100]
    y = y_test[100]

    predict(model, X, y)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 130, 64)           19968     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 57,802
Trainable params: 57,802
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
188/188 [==============================] - 25s 117ms/step - loss: 2.2143 - accuracy: 0.1739 - val_loss: 2.0